# Aggregate swaps

This notebook shows a method to get all Bancor trades over a time period.

Set the timestamps and sum the trade volume to get 24h volume, daily trading volume, etc.

This is not a suitable method over larger timeframes.


## Setup
---

In [1]:
from graphqlclient import GraphQLClient
import pandas as pd
import json
from pandas.io.json import json_normalize
from datetime import datetime

In [2]:
ENDPOINT = "https://api.thegraph.com/subgraphs/name/blocklytics/bancor"
client = GraphQLClient(ENDPOINT)

In [3]:
# Timestamps
start_time = int(datetime(2020,1,1).timestamp()) # Inclusive
end_time = int(datetime(2020,1,14).timestamp())   # Exclusive

## Fetch data
---

In [4]:
# Start with an empty df
df = pd.DataFrame()

# Results must be paginated
limit = 100
offset = 0
fetching_results = True

In [5]:
# Fetch paginated results
while fetching_results:
    QUERY = """
    {{
      swaps(
        first: {0}, 
        skip: {1},
        where:{{ 
          timestamp_gte:"{2}",
          timestamp_lt:"{3}"
        }},
        orderBy:timestamp,
        orderDirection:desc
      ) {{
        fromToken {{ id, symbol, decimals }}
        toToken {{ id, symbol, decimals }}
        amountPurchased
        amountReturned
        converterUsed {{ id }}
        timestamp
        transaction {{ id }}
      }}
    }}
    """.format(limit, offset, start_time, end_time)

    result = json.loads(client.execute(QUERY))
    _df = pd.DataFrame.from_records(json_normalize(result['data']['swaps']))
    
    # If df is empty, create it from new _df
    # Else append new _df to df
    if len(df.index) == 0:
        df = _df
    else:
        df = df.append(_df)
    
    # Prepare for pagination
    result_length = len(_df.index)
    if limit == result_length:
        offset += limit
    else:
        fetching_results = False

df.reset_index(drop=True, inplace=True)
df.sort_values('timestamp', ascending=False)
df.shape

(16894, 11)

# Format data
---
Convert wei values

In [6]:
df['amountPurchased_float'] = df.apply(lambda row: float(row['amountPurchased']) / 10 ** row['fromToken.decimals'], axis=1)
df['amountReturned_float'] = df.apply(lambda row: float(row['amountReturned']) / 10 ** row['toToken.decimals'], axis=1)

Get volume in base token terms (USDB or BNT)

In [7]:
df['volume_bnt'] = \
    df.apply(lambda row: row['amountPurchased_float'] if row['fromToken.id'] == "0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c" else 0, axis=1)\
    + df.apply(lambda row: row['amountReturned_float'] if row['toToken.id'] == "0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c" else 0, axis=1)

df['volume_usdb'] = \
    df.apply(lambda row: row['amountPurchased_float'] if row['fromToken.id'] == "0x309627af60f0926daa6041b8279484312f2bf060" else 0, axis=1)\
    + df.apply(lambda row: row['amountReturned_float'] if row['toToken.id'] == "0x309627af60f0926daa6041b8279484312f2bf060" else 0, axis=1)\


Convert timestamp

In [8]:
df['timestamp'] = df.apply(lambda row: datetime.fromtimestamp(int(row['timestamp'])), axis=1)

Preview swaps

In [9]:
df[['transaction.id', 'timestamp', 'amountPurchased_float', 'fromToken.symbol', 'amountReturned_float', 'toToken.symbol', 'volume_bnt', 'volume_usdb']]\
    .head(5)


,transaction.id,timestamp,amountPurchased_float,fromToken.symbol,amountReturned_float,toToken.symbol,volume_bnt,volume_usdb
0,0x1bece952ffed4ccea833343484edc673408ae36d46cb...,2020-01-13 23:58:50,0.036040,ETH,23.856345,BNT,23.856345,0.00000
1,0x1bece952ffed4ccea833343484edc673408ae36d46cb...,2020-01-13 23:58:50,23.856345,BNT,5.118950,USDB,23.856345,5.11895
2,0x1bece952ffed4ccea833343484edc673408ae36d46cb...,2020-01-13 23:58:50,5.118950,USDB,112.120597,REN,0.000000,5.11895
3,0x7750603ac38b22c01b8ef93f3519a5f88e585acb36c4...,2020-01-13 23:54:15,14546.000000,FXC,212.786256,BNT,212.786256,0.00000
4,0x7750603ac38b22c01b8ef93f3519a5f88e585acb36c4...,2020-01-13 23:54:15,212.786256,BNT,0.320178,ETH,212.786256,0.00000


# Volume per day
---

Add date column

In [10]:
df['date'] = df['timestamp'].apply(lambda row: row.date())

Add adj USDB column to avoid double-counting volume from BNT-USDB converter.

In [17]:
df['volume_usdb_adj'] = df[df['volume_bnt'] == 0]['volume_usdb']

## BNT

This does not include volume through USDB converters and excludes Smart Token -> Token trades.

In [18]:
df.groupby(by='date')[['volume_bnt', 'volume_usdb_adj']].sum()

,volume_bnt,volume_usdb_adj
date,,
2020-01-01,3.168038e+06,3932.857926
2020-01-02,4.289466e+06,9148.456023
2020-01-03,9.264600e+05,26382.312010
2020-01-04,6.092817e+05,1612.721415
2020-01-05,6.741570e+05,1137.642456
2020-01-06,9.345999e+05,2624.781718
2020-01-07,9.274317e+05,2997.933153
2020-01-08,8.372953e+05,833.055158
2020-01-09,7.526183e+05,1025.784478


# Volumes over period
---
## BNT Volume

In [12]:
df['volume_bnt'].sum()

15946826.561210565

## USDB Volume

Avoid double-counting volume by excluding BNT volume already counted above.

In [13]:
# df['volume_usdb'].sum() # This will double-count BNT volume from the BNT-USDB converter.

In [14]:
df[df['volume_bnt'] == 0]['volume_usdb'].sum() # This will not double-count BNT volume.

56369.43649855991

## Trades not counted

These trades were not counted as they did not include a base token of BNT or USDB.

In [19]:
df[(df['volume_bnt'] == 0) & (df['volume_usdb'] == 0)]

,amountPurchased,amountReturned,converterUsed.id,fromToken.decimals,fromToken.id,fromToken.symbol,timestamp,toToken.decimals,toToken.id,toToken.symbol,transaction.id,amountPurchased_float,amountReturned_float,volume_bnt,volume_usdb,date,volume_usdb_adj
86,1300000000000000000000,276510682924207719976,0x08b61ded2f558071fbdb827715e7aef16e76dd4f,18.0,0xbf2179859fc6d5bee9bf9158632dc51678a4100e,ELF,2020-01-13 20:43:26,18.0,0x0f2318565f1996cb1ed2f88e172135791bc1fcbf,ELFBNT,0xe04b8c5ecc8a66623ec0cac2f6761135b03370d9b0b8...,1300.000000,276.510683,0.0,0.0,2020-01-13,0.0
184,672982338899999543762,18005189976716193924,0x66540a3fcd929774a8dab59d56fe7a2d3538450f,18.0,0x8a9c67fee641579deba04928c4bc45f66e26343a,JRT,2020-01-13 19:28:48,18.0,0x4827e558e642861cd7a1c8f011b2b4661f8d51fa,JRTUSDB,0x6be03f358922d60f9ab6572ee448c5c54f3c9dfa5b68...,672.982339,18.005190,0.0,0.0,2020-01-13,0.0
420,140584404172766807559,14056985391290694690,0x3cfd18f931d449405ddc26e6b5b6b90f181f5bb9,18.0,0xe6b31fb3f29fbde1b92794b0867a315ff605a324,BNBBNT,2020-01-13 13:14:01,18.0,0xb8c77482e45f1f44de1745f52c74426c631bdd52,BNB,0x5c67abdec760872f9ab9b9565b88baf6e650033f1956...,140.584404,14.056985,0.0,0.0,2020-01-13,0.0
526,1671917584561820136384,2989761070234268274,0xd3ec78814966ca1eb4c923af4da86bf7e6c743ba,18.0,0xb1cd6e4153b2a390cf00a6556b0fc1458c4a5533,ETHBNT,2020-01-13 10:17:24,18.0,0xc0829421c1d260bd3cb3e0f06cfe2d52db2ce315,ETH,0xb031c8449008f00a345554ea2990c7bb4294702c3da2...,1671.917585,2.989761,0.0,0.0,2020-01-13,0.0
769,9790000000000000000000,263484834873414151996,0x66540a3fcd929774a8dab59d56fe7a2d3538450f,18.0,0x8a9c67fee641579deba04928c4bc45f66e26343a,JRT,2020-01-13 04:27:02,18.0,0x4827e558e642861cd7a1c8f011b2b4661f8d51fa,JRTUSDB,0xdd57545746662955bbf0304dc0da6cbcea41af541839...,9790.000000,263.484835,0.0,0.0,2020-01-13,0.0
1006,99337124713959782131,177075294181655180,0xd3ec78814966ca1eb4c923af4da86bf7e6c743ba,18.0,0xb1cd6e4153b2a390cf00a6556b0fc1458c4a5533,ETHBNT,2020-01-12 23:22:50,18.0,0xc0829421c1d260bd3cb3e0f06cfe2d52db2ce315,ETH,0x50cbf126dd5cad6a7df6b01e19c99f84d506cf01ad68...,99.337125,0.177075,0.0,0.0,2020-01-12,0.0
1094,1046028666222408726984,1865259425633494155,0xd3ec78814966ca1eb4c923af4da86bf7e6c743ba,18.0,0xb1cd6e4153b2a390cf00a6556b0fc1458c4a5533,ETHBNT,2020-01-12 22:28:54,18.0,0xc0829421c1d260bd3cb3e0f06cfe2d52db2ce315,ETH,0xdc1d11b11de812bf4973303a7b961456a9b2d7787475...,1046.028666,1.865259,0.0,0.0,2020-01-12,0.0
1483,700000000000000000,391844177080981104760,0xd3ec78814966ca1eb4c923af4da86bf7e6c743ba,18.0,0xc0829421c1d260bd3cb3e0f06cfe2d52db2ce315,ETH,2020-01-12 16:58:17,18.0,0xb1cd6e4153b2a390cf00a6556b0fc1458c4a5533,ETHBNT,0x35efc67ab1d06c00a75f82612d4aaaca31b37aba1450...,0.700000,391.844177,0.0,0.0,2020-01-12,0.0
1499,244777804002679363853,436529730662718360,0xd3ec78814966ca1eb4c923af4da86bf7e6c743ba,18.0,0xb1cd6e4153b2a390cf00a6556b0fc1458c4a5533,ETHBNT,2020-01-12 16:24:11,18.0,0xc0829421c1d260bd3cb3e0f06cfe2d52db2ce315,ETH,0x5b8169464e44d837e094a27e269ffb47e612558608fe...,244.777804,0.436530,0.0,0.0,2020-01-12,0.0
1500,627358516900000164929,16577527983926330591,0x66540a3fcd929774a8dab59d56fe7a2d3538450f,18.0,0x8a9c67fee641579deba04928c4bc45f66e26343a,JRT,2020-01-12 16:20:34,18.0,0x4827e558e642861cd7a1c8f011b2b4661f8d51fa,JRTUSDB,0x7a138e454083f368a666eefd29403197fa97382214be...,627.358517,16.577528,0.0,0.0,2020-01-12,0.0


In [20]:
df.iloc[86]

amountPurchased                                     1300000000000000000000
amountReturned                                       276510682924207719976
converterUsed.id                0x08b61ded2f558071fbdb827715e7aef16e76dd4f
fromToken.decimals                                                      18
fromToken.id                    0xbf2179859fc6d5bee9bf9158632dc51678a4100e
fromToken.symbol                                                       ELF
timestamp                                              2020-01-13 20:43:26
toToken.decimals                                                        18
toToken.id                      0x0f2318565f1996cb1ed2f88e172135791bc1fcbf
toToken.symbol                                                      ELFBNT
transaction.id           0xe04b8c5ecc8a66623ec0cac2f6761135b03370d9b0b8...
amountPurchased_float                                                 1300
amountReturned_float                                               276.511
volume_bnt               